In [86]:
#!g1.1
from config import config
import pickle
with open(config['wikidata']['transe'], "rb") as f:
    transe = pickle.load(f)
import json
with open(config['dataset']['postprocess']['train'], 'r') as f:
    el_single_results = json.load(f)

In [87]:
#!g1.1
%load_ext autoreload
%autoreload 2
from model_utils.thswad import THWADModel, load_model_from_transe
from model_utils.trainer import THSWADTrainer
from utils.ds import *

In [3]:
#!g1.1
ds_ = DialogDataset(el_single_results['entity_ids'], 'pad', 5)
items2id = {}
for i, (prev, next_id) in enumerate(ds_):
    for item in prev:
        if item not in items2id and item != 'pad':
            items2id[item] = len(items2id)
    if next_id not in items2id and next_id != 'pad':
        items2id[next_id] = len(items2id)
id2items = list(items2id.keys())
ds = DialogDataset(el_single_results['entity_ids'], len(items2id), 5, items2id)
ds[0]

/home/jupyter/work/resources/dream-entity-recommendation/utils/ds.py:64: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if np.all(np.array(prev) == pad_item):


(array([3177, 3177, 3177, 3177,    0]), 1)

In [20]:
#!g1.1
wds = load_wiki_dataset(config['wikidata']['dataset']['train'], transe.graph)

In [21]:
#!g1.1
from torch.utils.data import DataLoader

BS = 256

dl = DataLoader(ds, batch_size=BS, shuffle=False)
wdl = DataLoader(wds, batch_size=BS, shuffle=False)
train_loader = TSHWADLoader(wdl, dl)

In [88]:
#!g1.1
model = load_model_from_transe(transe, id2items, prev_items_total=5)
model.load('model_checkpoints/model.pt')

NameError: name 'id2items' is not defined

In [22]:
#!g1.1
opt = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(opt, 1, gamma=0.9)

trainer = THSWADTrainer(
    margin=1.,
    norm_lambda=1.,
    clipping_max_value=1.,
    model_target=1.,
    kg_lambda=.4,
    model=model,
    optimizer=opt,
    scheduler=scheduler,
    train_data_loader=train_loader,
    val_data_loader=train_loader,
    test_data_loader=train_loader,
    verbose=False,
)

In [23]:
#!g1.1
trainer.train(5)

Training:  99%|█████████▉| 158870/160794 [19:18<00:14, 137.08it/s]


In [24]:
#!g1.1
BS = 32

dl_v = DataLoader(ds, batch_size=BS, shuffle=False)
wdl_v = DataLoader([], batch_size=BS, shuffle=False)
val_loader = TSHWADLoader(wdl, dl)

trainer.test(val_loader)

Validating:  99%|█████████▉| 158870/160794 [5:18:32<03:51,  8.31it/s]


{'accuracy': tensor(0.0033, device='cuda:0')}

In [ ]:
#!g1.1
model.prev_meaner

In [ ]:
#!g1.1
len(model.item2entity)

In [20]:
#!g1.1
! head -n 1000000 dataset/wikidata5m_transductive_train.txt > dataset/wikidata5m_transductive_train_head.txt

In [ ]:
#!g1.1
